In [1]:
from Classes.Arquivo import Arquivo
from Classes.BancoDados import BancoDados

In [2]:
try:
    #Criação dos dataframes
    s_total = Arquivo("Arquivos CSV/suicidios-total.csv")
    homens_arma = Arquivo("Arquivos CSV/suicidios-homens-armas-fogo.csv")
    mulheres_arma = Arquivo("Arquivos CSV/suicidios-mulheres-armas-fogo.csv")
    s_homens = Arquivo("Arquivos CSV/suicidios-homens.csv")
    s_mulheres = Arquivo("Arquivos CSV/suicidios-mulheres.csv")
    s_regiao = Arquivo("Arquivos CSV/suicidios-região.csv")

    #Inspeção
    if s_total.verifica and homens_arma.verifica and mulheres_arma.verifica and s_homens.verifica and s_mulheres.verifica and s_regiao.verifica:
        print("Arquivos CSV inspecionados com sucesso!")
    else:
        print(f"""
Status de verificação de cada arquivo:
suicidios-total.csv = {s_total.verifica}
suicidios-homens-armas-fogo.csv = {homens_arma.verifica}
suicidios-mulheres-armas-fogo.csv = {mulheres_arma.verifica}
suicidios-homens.csv = {s_homens.verifica}
suicidios-mulheres.csv = {s_mulheres.verifica}
suicidios-região.csv = {s_regiao.verifica}
        """)
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivos CSV inspecionados com sucesso!


In [3]:
# 
BancoLocal = BancoDados('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')
print(dir(BancoLocal))

Conexão bem-sucedida!
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'buscar_dados', 'criacao_tabelas', 'db_connect', 'encerrar', 'inserir_gen', 'inserir_total']


In [ ]:
BancoLocal.criacao_tabelas()

In [ ]:
BancoLocal.inserir_total(s_total.dataframe)

In [4]:
colunasPeriodo = BancoLocal.buscar_dados('periodo')[['per_cod', 'ano']]

#1 Feminino
#2 Masculino
colunasGenero = BancoLocal.buscar_dados('genero')['gen_cod']


colunasRegiao = BancoLocal.buscar_dados('regiao')[['reg_cod', 'nome']]

BancoLocal.inserir_gen(s_homens.dataframe, colunasPeriodo)

'Erro ao inserir os dados: "[\'per_cod\', \'gen_cod\'] not in index"'